# Tutorial about localization precision

Localization precision is determined from consecutive localizations that are identified within a certain search radius. The results include the distances between localization pairs, the position deltas for each coordinate and the corresponding frame of the first localizations.

In [ ]:
from pathlib import Path

%matplotlib inline

import matplotlib.pyplot as plt

import locan as lc

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

In [ ]:
# A path in which test data can be found:
TEST_DIR: Path = Path.cwd().parents[2] / "tests"
TEST_DIR

## Load SMLM data file

Identify some data in the test_data directory and provide a path using `pathlib.Path` (returned by `lc.ROOT_DIR`)

In [ ]:
path = TEST_DIR / 'test_data/npc_gp210.asdf'
print(path, '\n')
dat = lc.load_locdata(path=path, file_type=lc.FileType.ASDF)

Print information about the data: 

In [ ]:
print(dat.data.head(), '\n')
print('Summary:')
dat.print_summary()
print('Properties:')
print(dat.properties)

## Visualization

In [ ]:
lc.render_2d(dat, bin_size=10, rescale=(0,100));

## Analyze localization precision

In [ ]:
lp = lc.LocalizationPrecision(radius=50)

In [ ]:
lp.compute(dat)
lp.results.head()

The plot

In [ ]:
lp.plot(loc_property='position_delta_x', window=10);

In [ ]:
lp.plot(window=10);

The histogram for the distances per default includes a fit to a distribution expected for normal distributed localizations. Sigma / sqrt(2) is the localization precision.

In [ ]:
lp.hist();

Alternatively the position deltas can be histogrammed.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
lp.hist(ax=ax[0], loc_property='position_delta_x')
lp.hist(ax=ax[1], loc_property='position_delta_y')
plt.tight_layout()
plt.show()

## Fit distributions and show parameters

Appropriate distribution functions are fitted to the data either by calling the hist function or by running:

In [ ]:
lp.fit_distributions()

The estimated fit parameters are provided under the `distribution_statistics` attribute

In [ ]:
lp.distribution_statistics.parameter_dict()

Remember: localization precision is typically defined as the standard deviation for the distances between localizations and their center position (the true dye position). Therefore, the estimated sigmas have to be divided by sqrt(2) to yield localization precision.